In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select # type: ignore
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, TimeoutException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
#from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.service import Service
import time
import requests  # For making HTTP requests
from bs4 import BeautifulSoup  # For parsing HTML content
from fake_useragent import UserAgent  # For generating random user agents
import pandas as pd  # For data manipulation and creating DataFrames
import numpy as np
import concurrent.futures
from selenium.webdriver.common.action_chains import ActionChains

In [6]:
service = Service(executable_path=r'C:\Users\DELL\Documents\web_scraping\chromedriver-win64\chromedriver.exe')
driver = webdriver.Chrome(service=service)
driver.get("https://fzmovies.net/")
driver.implicitly_wait(10)
all_data = []


try:
    
    getting_250_list = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//ul//li//a[contains(text(),'IMDB Top 250 movies')]"))

    )
    getting_250_list.click()
    time.sleep(3)

    while True:
        tables = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, "table"))
        )

        for table in tables:
            rows = table.find_elements(By.TAG_NAME, "tr")
            for row in rows[1:]: 
                second_td = row.find_elements(By.TAG_NAME, "td")[1]

                span = second_td.find_element(By.TAG_NAME, "span")
                a_element = span.find_element(By.TAG_NAME, "a")
                b_element = a_element.find_element(By.TAG_NAME, "b")
                small_tags_in_span = span.find_elements(By.TAG_NAME, "small")

                small_tags_in_td = second_td.find_elements(By.TAG_NAME, "small")
                rating_small = small_tags_in_td[0].find_elements(By.TAG_NAME, "span")  

                data = pd.DataFrame({
                "Movie Name": b_element.text,
                "small_tag_text": small_tags_in_span[0].text,
                "Description": small_tags_in_span[2].text if len(small_tags_in_span) > 2 else None,  
                "IMBD Rating": rating_small[0].text if rating_small else None, 
                "IMDB Votes": rating_small[1].text if len(rating_small) > 1 else None,  
            })

                

        try:
            WebDriverWait(driver, 10).until(
        EC.invisibility_of_element_located(
            (By.CSS_SELECTOR, 
             "a[href*='haychalk.com'], "
             "a[href*='achieveweakness.com'], "
             "a[href*='flusoprano.com'], "
             "a[href*='tributeparticle.com'], "
             "iframe[id^='container-']"  
            )
        )
    )
            next_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div//small//a[contains(text(),'Next')]"))
    )
            current_url = driver.current_url
            try:
                next_button.click() 
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].click();", next_button)


            WebDriverWait(driver, 10).until(EC.url_changes(current_url))

        except NoSuchElementException:
            break

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    print("Web scraping completed.")

    
    driver.quit()  

An error occurred: Message: 

Web scraping completed.
